In [1]:
import os, glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import cartopy
import cartopy.io
import cartopy.io.img_tiles
import cartopy.feature
import shapely

import functions

## Define Variables

In [2]:
skip_ncgen_prcp = True # set 'False' for the first run to save temporary processed netcdf files
skip_ncgen_temp = True

path_station = 'HEROES'
station_folder = {
    'Baylling': 'Weather station data at Baylling School (Trashiyangtse)',
    'Bjishong': 'Weather station data at Bjishong School (Gasa, Damji)',
    'Damthang': 'Weather station data at Damthang School (Haa) ',
    'Galing': 'Weather station data at Galing School (Trashigang)',
    'Gyelpozhing': 'Weather station data at Gyelpozhing School (Mongar)',
    'Jakar': 'Weather station data at Jakar School (Bumthang)',
    'Jomotsangkha': 'Weather station data at Jomotsangkha School (Samdrupjongkhar)',
    'Khasadrapchu': 'Weather station data at Khasadrapchu School (Thimphu)',
    'Nobding': 'Weather station data at Nobding School (Wangdiphodrang)',
    'Pelrithang': 'Weather station data at Pelrithang School (Gelephu)',
    'Sakteng': 'Weather station data at Sakteng School (Trashigang)',
    'Tengmachu': 'Weather station data at Tangmachu School (Lhuntse)',
    'Tendruk': 'Weather station data at Tendruk School (Samtse)',
    'Yurung': 'Weather station data at Yurung School (Samtse)',
}
station_lonlat = {
    'Baylling': [91.5035, 27.6213],
    'Bjishong': [89.7267, 27.809],
    'Damthang': [89.2033, 27.4342],
    'Galing': [91.6405, 27.339],
    'Gyelpozhing': [91.1935, 27.2369],
    'Jakar': [90.7376, 27.5458],
    'Jomotsangkha': [92.1097, 26.8962],
    'Khasadrapchu': [89.5803, 27.3855],
    'Nobding': [90.1504, 27.5469],
    'Pelrithang': [90.4913, 26.9021],
    'Sakteng': [91.923, 27.4031],
    'Tengmachu': [91.1914, 27.5922],
    'Tendruk': [88.8722, 27.1211],
    'Yurung': [91.348, 27.0215],
}

bounds = [88.6875, 25.5, 93.375, 28.625] # covers the basin over Bhutan
bounds_bhutan = [88.6875, 26.6875, 92.125, 28.25] # covers Bhutan
date_range = ['2015-01-01','2019-12-31'] # for statistics

path_data = '/rcfs/projects/sage/hydro-modeling/datasets'
#path_data = 'datasets' # if there is a symlink created for the absolute path above
list_models = ['ERA5', 'ERA5-L', 'MSWX-P', 'GSWP3-W5E5', 'HMA', 'GMFD'] # a list of model variables
years_load = np.arange(2015, 2021) # for loading datasets
path_temps, path_figs = 'tempfiles', 'figures'
os.makedirs(path_temps, exist_ok = True)
os.makedirs(path_figs, exist_ok = True)

vars_list = {
    'ERA5': [
        't2m', # 2 metre temperature
        'tp', # Total precipitation
    ],
    'ERA5-L': [
        't2m', # 2 metre temperature
        'tp', # Total precipitation
    ],
    'MSWX-P': [
        'air_temperature',
        'air_temperature_max',
        'air_temperature_min',
        'precipitation',
    ],
    'GSWP3-W5E5': [
        'pr', # Precipitation
        'tas', # Near-Surface Air Temperature
        'tasmax', # Near-Surface Air Temperature
        'tasmin', # Near-Surface Air Temperature
    ],
    'HMA': [
        'P_CHP', # Downscaled CHIRPS total precipitation
        'P_ECMWF', # Downscaled ECMWF total precipitation
        'Tair', # Downscaled ECMWF near-surface temperature
    ],
    'GMFD': [
        'prcp', # Precipitation
        'tas', # Air temperature
        'tmax', # Maximum air temperature
        'tmin', # Minimum air temperature
    ],
}
key_prcp = {'ERA5': ['tp'], 'ERA5-L': ['tp'], 'MSWX-P': ['precipitation'], 'GSWP3-W5E5': ['pr'], 'HMA': ['P_CHP', 'P_ECMWF'], 'GMFD': ['prcp']} # preipitation variables in each model
key_temp = {'ERA5': 't2m', 'ERA5-L': 't2m', 'HMA': 'Tair'} # sub-daily temperature variables in each model
key_tmin = {'MSWX-P': 'air_temperature_min', 'GSWP3-W5E5': 'tasmin', 'GMFD': 'tmin'} # daily minimum temperature variables in each model
key_tmax = {'MSWX-P': 'air_temperature_max', 'GSWP3-W5E5': 'tasmax', 'GMFD': 'tmax'} # daily maximum temperature variables in each model

## Load Datasets

In [3]:
dict_df = {}
for key, folder in station_folder.items():
    df = pd.read_csv(glob.glob(os.path.join(path_station, folder, 'data', '*_data_monthly.csv'))[0])
    df.index = pd.to_datetime(df['Month'].astype(str) + '/1/' + df['Year'].astype(str))
    dict_df[key] = df

In [4]:
list_ds = []
if not skip_ncgen_prcp or not skip_ncgen_temp:
    for m in list_models:
        ds = functions.read_datasets(path = path_data, model = m, years = years_load, bounds = bounds, vars_list = vars_list[m], save_nc = None) # load datasets to process
        for key in key_prcp[m]: ds[key] = functions.convert_precipitation(da_prcp = ds[key], model = m) # convert a pricipitation variable into total precipitation per step
        if m in key_temp.keys(): ds[key_temp[m]] = functions.convert_temperature(da_temp = ds[key_temp[m]], model = m) # convert a sub-daily temperature variable into degreeC
        if m in key_tmin.keys(): ds[key_tmin[m]] = functions.convert_temperature(da_temp = ds[key_tmin[m]], model = m) # convert a daily minimum temperature variable into degreeC
        if m in key_tmax.keys(): ds[key_tmax[m]] = functions.convert_temperature(da_temp = ds[key_tmax[m]], model = m) # convert a daily maximum temperature variable into degreeC
        list_ds.append(ds)

In [5]:
list_name_prcp, list_da_mprcp = [], []
if not skip_ncgen_prcp:
    for i, m in enumerate(list_models):
        for key in key_prcp[m]:
            list_name_prcp.append(f'{functions.models_name[m]}: "{key}"')
            da = list_ds[i][key].resample(time = '1MS', skipna = False).sum() # Monthly Precipitation Total
            date_min = max(pd.Timestamp(date_range[0]), pd.Timestamp(da['time'][0].values))
            date_max = min(pd.Timestamp(date_range[-1]), pd.Timestamp(da['time'][-1].values))
            da_mprcp = da.sel(time = pd.date_range(date_min, date_max, freq = '1MS')).load()
            da_mprcp.to_netcdf(os.path.join(path_temps, f'_ncgen_HEROES_prcp_{m}_{key}.nc'))
            list_da_mprcp.append(da_mprcp)
else:
    for i, m in enumerate(list_models):
        for key in key_prcp[m]:
            list_name_prcp.append(f'{functions.models_name[m]}: "{key}"')
            list_da_mprcp.append(xr.load_dataarray(os.path.join(path_temps, f'_ncgen_HEROES_prcp_{m}_{key}.nc'), decode_coords = 'all'))

list_name_temp, list_da_mtmean, list_da_mtmin, list_da_mtmax = [], [], [], []
if not skip_ncgen_temp:
    for i, m in enumerate(list_models):
        if m in key_temp.keys():
            list_name_temp.append(f'{functions.models_name[m]}: "{key_temp[m]}"')
            da = list_ds[i][key_temp[m]].resample(time = '1MS', skipna = False).mean() # Monthly Mean if the sub-hourly data are available
        elif m in key_tmin.keys() and m in key_tmax.keys():
            list_name_temp.append(f'{functions.models_name[m]}: "{key_tmin[m]}, {key_tmax[m]}"')
            da = (list_ds[i][key_tmin[m]] + list_ds[i][key_tmax[m]]) / 2  # Average of Tmax and Tmin if the sub-hourly data are unavailable, similar to the paper
            da = da.resample(time = '1MS', skipna = False).mean() # Monthly Mean
            da.name = key_tmin[m][0].replace('min','mean_by_minmax')
        date_min = max(pd.Timestamp(date_range[0]), pd.Timestamp(da['time'][0].values))
        date_max = min(pd.Timestamp(date_range[-1]), pd.Timestamp(da['time'][-1].values))
        da_mtmean = da.sel(time = pd.date_range(date_min, date_max, freq = '1MS')).load()
        da_mtmean.to_netcdf(os.path.join(path_temps, f'_ncgen_HEROES_tmean_{m}.nc'))
        list_da_mtmean.append(da_mtmean)

        if m in key_temp.keys(): da_min = list_ds[i][key_temp[m]].resample(time = '1D', skipna = False).min() # Daily Minimum Temperature
        elif m in key_tmin.keys() and m in key_tmax.keys(): da_min = list_ds[i][key_tmin[m]]
        date_min = max(pd.Timestamp(date_range[0]), pd.Timestamp(da_min['time'][0].values))
        date_max = min(pd.Timestamp(date_range[-1]), pd.Timestamp(da_min['time'][-1].values))
        da_mtmin = da_min.sel(time = pd.date_range(date_min, date_max, freq = '1D')).resample(time = '1MS', skipna = False).min() # Monthly Min
        da_mtmin.to_netcdf(os.path.join(path_temps, f'_ncgen_HEROES_tmin_{m}.nc'))
        list_da_mtmin.append(da_mtmin)
        
        if m in key_temp.keys(): da_max = list_ds[i][key_temp[m]].resample(time = '1D', skipna = False).max() # Daily Maximum Temperature
        elif m in key_tmin.keys() and m in key_tmax.keys(): da_max = list_ds[i][key_tmax[m]]
        date_min = max(pd.Timestamp(date_range[0]), pd.Timestamp(da_max['time'][0].values))
        date_max = min(pd.Timestamp(date_range[-1]), pd.Timestamp(da_max['time'][-1].values))
        da_mtmax = da_max.sel(time = pd.date_range(date_min, date_max, freq = '1D')).resample(time = '1MS', skipna = False).max() # Monthly Max
        da_mtmax.to_netcdf(os.path.join(path_temps, f'_ncgen_HEROES_tmax_{m}.nc'))
        list_da_mtmax.append(da_mtmax)
else:
    for i, m in enumerate(list_models):
        if m in key_temp.keys(): list_name_temp.append(f'{functions.models_name[m]}: "{key_temp[m]}"')
        elif m in key_tmin.keys() and m in key_tmax.keys(): list_name_temp.append(f'{functions.models_name[m]}: "{key_tmin[m]}, {key_tmax[m]}"')
        list_da_mtmean.append(xr.load_dataarray(os.path.join(path_temps, f'_ncgen_HEROES_tmean_{m}.nc'), decode_coords = 'all'))
        list_da_mtmin.append(xr.load_dataarray(os.path.join(path_temps, f'_ncgen_HEROES_tmin_{m}.nc'), decode_coords = 'all'))
        list_da_mtmax.append(xr.load_dataarray(os.path.join(path_temps, f'_ncgen_HEROES_tmax_{m}.nc'), decode_coords = 'all'))

## Set Plot Properties

In [6]:
shp_file = os.path.join('shapefiles', 'btn_admbnda_adm1_bnlc_20201026.shp') # Bhutan Administrative Boundaries (Level 1)
shp_file_mask = os.path.join('shapefiles', 'gridmask_bhutan.shp') # Polygon masking outside of Bhutan

baseMap = cartopy.io.img_tiles.GoogleTiles(style = 'satellite')

shape_feature = cartopy.feature.ShapelyFeature(cartopy.io.shapereader.Reader(shp_file).geometries(), crs = cartopy.crs.PlateCarree(), facecolor = 'none', linewidth = 0.5) # Bhutan Administrative Boundaries (Level 1)
shape_feature_mask = cartopy.feature.ShapelyFeature(cartopy.io.shapereader.Reader(shp_file_mask).geometries(), crs = cartopy.crs.PlateCarree(), facecolor = 'w', linewidth = 0.5) # masking outside of Bhutan
polygon_feature = shapely.unary_union(shapely.geometry.MultiPolygon(shape_feature.geometries())) # applying statistics only for Bhutan

# function to select the nearest grid
def select_da_nearest(da, lonlat):
    if 'lon' in da.dims and 'lat' in da.dims: x, y = 'lon', 'lat'
    elif 'longitude' in da.dims and 'latitude' in da.dims: x, y = 'longitude', 'latitude'
    elif 'x' in da.dims and 'y' in da.dims: x, y = 'x', 'y'
    else: return da
    return da.sel({x: lonlat[0], y: lonlat[1]}, method = 'nearest')

# function to plot a timeseries of variable
def plot_obs(df, df_name, list_da_sel, list_name, title = '', xlabel = '', ylabel = '', xlim = [None, None], ylim = [None, None], savefig = None, show = None):
    fig, ax = plt.subplots(figsize = (14, 4))
    for i, da_sel in enumerate(list_da_sel): ax.plot(da_sel['time'], da_sel, label = list_name[i])
    ax.plot(df.index, df, 'r-o', label = df_name)
    ax.set_title(title)
    ax.set_xlabel(xlabel); ax.set_ylabel(ylabel)
    ax.set_xlim(xlim); ax.set_ylim(ylim)
    ax.grid()

    fig.subplots_adjust(bottom = 0.15)
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, ncol = 4, loc = 'lower center', bbox_to_anchor = (0, 0.01, 1, 0.02))
    fig.tight_layout(rect = [0, 0.15, 1, 1])

    if savefig is not None: plt.savefig(savefig, bbox_inches = 'tight')
    if show is not None: plt.show()
    return

# function to create a masked dataarray for statistics
def create_da_masked(da, polygon = None):
    if polygon is None: return da
    if 'lon' in da.dims and 'lat' in da.dims: x, y = 'lon', 'lat'
    elif 'longitude' in da.dims and 'latitude' in da.dims: x, y = 'longitude', 'latitude'
    elif 'x' in da.dims and 'y' in da.dims: x, y = 'x', 'y'
    else: return da
    xs, ys = np.meshgrid(da[x], da[y])
    xys = np.stack([xs.flatten(), ys.flatten()], axis = -1)
    mask = shapely.contains_xy(polygon, xys)
    mask = mask.reshape([len(da[y]), len(da[x])])
    return da.where(mask == True)

# function to plot a geospatial map of variable, including observations
def plot_models(list_da, list_name, list_shapes, bounds, norm = None, cmap = 'nipy_spectral', suptitle = '', clabel = '', stats = False, stats_polygon = None, stats_unit = 'mm', savefig = None, show = True, obs = None):
    nplot = len(list_da)
    fig, axes = plt.subplots(nrows = (1 + nplot) // 2, ncols = 2, figsize = (10, 14), subplot_kw = {'projection': baseMap.crs})
    fig.suptitle(suptitle)
    for i, ax in enumerate(axes.flat):
        ax.set_extent([bounds[0], bounds[2], bounds[1], bounds[3]])
        fgl = ax.gridlines(crs = cartopy.crs.PlateCarree(), linestyle = '--', alpha = 0.25, draw_labels = True)
        fgl.top_labels, fgl.right_labels = False, False
        fgl.xlabel_style, fgl.ylabel_style = {'color': 'gray', 'size': 8}, {'color': 'gray', 'size': 8}

        if i < nplot:
            fda = list_da[i].plot(ax = ax, transform = cartopy.crs.PlateCarree(), norm = norm, cmap = cmap, add_colorbar = False)
            if obs is not None: ax.scatter(obs[:, 0], obs[:, 1], c = obs[:, 2], transform = cartopy.crs.PlateCarree(), norm = norm, cmap = cmap, edgecolors = 'k')
            
            if stats == True or stats_polygon is not None: # applying statistics only for Bhutan
                da_masked = create_da_masked(da = list_da[i], polygon = stats_polygon)
                stats = 'Mean: {:.0f} {}, Min: {:.0f} {}, Max: {:.0f} {}'.format(np.nanmean(da_masked), stats_unit, np.nanmin(da_masked), stats_unit, np.nanmax(da_masked), stats_unit)
                ax.set_title(list_name[i] + '\n' + stats)
            else: ax.set_title(list_name[i])
        else: ax.add_image(baseMap, 10)
        for s in list_shapes: ax.add_feature(s)
        
    fig.subplots_adjust(bottom = 0.05)
    cax = fig.add_axes([0.2, 0.025, 0.6, 0.02])
    fig.colorbar(fda, label = clabel, cax = cax, orientation = 'horizontal')
    fig.tight_layout(rect = [0, 0.05, 1, 1])

    if savefig is not None: plt.savefig(savefig, bbox_inches = 'tight')
    if show is not None: plt.show()
    return

# colormaps
precip_11lev = np.array([
    [255, 255, 255],
    [237, 250, 194],
    [205, 255, 205],
    [153, 240, 178],
    [83, 189, 159],
    [50, 166, 150],
    [50, 150, 180],
    [5, 112, 176],
    [5, 80, 140],
    [10, 31, 150],
    [44, 2, 70],
    [106, 44, 90],
])
precip_11lev = precip_11lev / 255

prcp_bounds = [0, 1, 2, 5, 10, 20, 200, 300, 500, 1000]
prcp_norm = matplotlib.colors.BoundaryNorm(boundaries = prcp_bounds, ncolors = 13, extend = 'max')

prcp_bounds_tminmax = np.linspace(-15, 30, 10, endpoint = True)
prcp_norm_tminmax = matplotlib.colors.BoundaryNorm(boundaries = prcp_bounds_tminmax, ncolors = 256, extend = 'both')

## Monthly Precipitation Total at Stations

In [7]:
# Monthly Precipitation Total at Stations
for key, df in dict_df.items():
    if 'Rainfall' in df.columns:
        df.loc[df['Rainfall'] == '#VALUE!', 'Rainfall'] = np.nan
        plot_obs(df = df['Rainfall'].astype(float), df_name = '{} School'.format(key),
                list_da_sel = [select_da_nearest(da, station_lonlat[key]) for da in list_da_mprcp], list_name = list_name_prcp,
                title = 'Monthly Precipitation Total at {} School (Lon: {:.3f} deg, Lat: {:.3f} deg)'.format(key, station_lonlat[key][0], station_lonlat[key][1]), xlabel = 'Time', ylabel = 'Precipitation [mm]', xlim = [None, None], ylim = [None, None],
                savefig = os.path.join(path_figs, f'HEROES-PRCP-STATION-{key}.png'), show = None)
        plt.close('all')

## Monthly Mean Temperature at Stations

In [8]:
# Monthly Mean Temperature at Stations
for key, df in dict_df.items():
    if 'Tmean' in df.columns:
        df.loc[df['Tmean'] == '#VALUE!', 'Tmean'] = np.nan
        plot_obs(df = df['Tmean'].astype(float), df_name = '{} School'.format(key),
                list_da_sel = [select_da_nearest(da, station_lonlat[key]) for da in list_da_mtmean], list_name = list_name_temp,
                title = 'Monthly Mean Temperature at {} School (Lon: {:.3f} deg, Lat: {:.3f} deg)'.format(key, station_lonlat[key][0], station_lonlat[key][1]), xlabel = 'Time', ylabel = 'Temperature [degC]', xlim = [None, None], ylim = [None, None],
                savefig = os.path.join(path_figs, f'HEROES-TMEAN-STATION-{key}.png'), show = None)
        plt.close('all')

## Monthly Minimum Temperature at Stations

In [9]:
# Monthly Minimum Temperature at Stations
for key, df in dict_df.items():
    if 'Tmin' in df.columns:
        df.loc[df['Tmin'] == '#VALUE!', 'Tmin'] = np.nan
        plot_obs(df = df['Tmin'].astype(float), df_name = '{} School'.format(key),
                list_da_sel = [select_da_nearest(da, station_lonlat[key]) for da in list_da_mtmin], list_name = list_name_temp,
                title = 'Monthly Minimum Temperature at {} School (Lon: {:.3f} deg, Lat: {:.3f} deg)'.format(key, station_lonlat[key][0], station_lonlat[key][1]), xlabel = 'Time', ylabel = 'Temperature [degC]', xlim = [None, None], ylim = [None, None],
                savefig = os.path.join(path_figs, f'HEROES-TMIN-STATION-{key}.png'), show = None)
        plt.close('all')

## Monthly Maximum Temperature at Stations

In [10]:
# Monthly Maximum Temperature at Stations
for key, df in dict_df.items():
    if 'Tmax' in df.columns:
        df.loc[df['Tmax'] == '#VALUE!', 'Tmax'] = np.nan
        plot_obs(df = df['Tmax'].astype(float), df_name = '{} School'.format(key),
                list_da_sel = [select_da_nearest(da, station_lonlat[key]) for da in list_da_mtmax], list_name = list_name_temp,
                title = 'Monthly Maximum Temperature at {} School (Lon: {:.3f} deg, Lat: {:.3f} deg)'.format(key, station_lonlat[key][0], station_lonlat[key][1]), xlabel = 'Time', ylabel = 'Temperature [degC]', xlim = [None, None], ylim = [None, None],
                savefig = os.path.join(path_figs, f'HEROES-TMAX-STATION-{key}.png'), show = None)
        plt.close('all')

## Monthly Precipitation Total

In [11]:
# Monthly Precipitation Total (Bhutan) w/ statistics
for month in pd.date_range(date_range[0], date_range[1], freq = '1MS'):
    obs = []
    for df in dict_df.values():
        try: obs.append(float(df['Rainfall'].loc[month]))
        except: obs.append(np.inf)
    obs = np.concatenate([np.stack(list(station_lonlat.values())), np.array(obs)[:, None]], axis = 1)
    list_da = []
    for da in list_da_mprcp:
        if month in da['time']: list_da.append(da.sel(time = month))
        else: list_da.append(da.isel(time = 0) * np.inf)
    
    plot_models(list_da = list_da, list_name = list_name_prcp,
                list_shapes = [shape_feature, shape_feature_mask], bounds = bounds_bhutan,
                norm = prcp_norm, cmap = matplotlib.colors.ListedColormap(precip_11lev), suptitle = month.strftime('%Y-%m'), clabel = 'Monthly Precipitation Total [mm]',
                stats = True, stats_polygon = polygon_feature, stats_unit = 'mm',
                savefig = os.path.join(path_figs, f'HEROES-PRCP-{month.strftime("%Y-%m")}-Bhutan.png'), show = None,
                obs = obs)
    plt.close('all')

/tmp/ipykernel_7881/265708999.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/265708999.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/265708999.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/265708999.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/265708999.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/265708999.py:76

## Modify Plot Properties for Temperature

In [12]:
# function to plot a geospatial map of variable, including observations
def plot_models(list_da, list_name, list_shapes, bounds, norm = None, cmap = 'nipy_spectral', suptitle = '', clabel = '', stats = False, stats_polygon = None, stats_unit = 'mm', savefig = None, show = True, obs = None):
    nplot = len(list_da)
    fig, axes = plt.subplots(nrows = (1 + nplot) // 2, ncols = 2, figsize = (10, 10), subplot_kw = {'projection': baseMap.crs})
    fig.suptitle(suptitle)
    for i, ax in enumerate(axes.flat):
        ax.set_extent([bounds[0], bounds[2], bounds[1], bounds[3]])
        fgl = ax.gridlines(crs = cartopy.crs.PlateCarree(), linestyle = '--', alpha = 0.25, draw_labels = True)
        fgl.top_labels, fgl.right_labels = False, False
        fgl.xlabel_style, fgl.ylabel_style = {'color': 'gray', 'size': 8}, {'color': 'gray', 'size': 8}

        if i < nplot:
            fda = list_da[i].plot(ax = ax, transform = cartopy.crs.PlateCarree(), norm = norm, cmap = cmap, add_colorbar = False)
            if obs is not None: ax.scatter(obs[:, 0], obs[:, 1], c = obs[:, 2], transform = cartopy.crs.PlateCarree(), norm = norm, cmap = cmap, edgecolors = 'k')
            
            if stats == True or stats_polygon is not None: # applying statistics only for Bhutan
                da_masked = create_da_masked(da = list_da[i], polygon = stats_polygon)
                stats = 'Mean: {:.0f} {}, Min: {:.0f} {}, Max: {:.0f} {}'.format(np.nanmean(da_masked), stats_unit, np.nanmin(da_masked), stats_unit, np.nanmax(da_masked), stats_unit)
                ax.set_title(list_name[i] + '\n' + stats)
            else: ax.set_title(list_name[i])
        else: ax.add_image(baseMap, 10)
        for s in list_shapes: ax.add_feature(s)
        
    fig.subplots_adjust(bottom = 0.05)
    cax = fig.add_axes([0.2, 0.025, 0.6, 0.02])
    fig.colorbar(fda, label = clabel, cax = cax, orientation = 'horizontal')
    fig.tight_layout(rect = [0, 0.05, 1, 1])

    if savefig is not None: plt.savefig(savefig, bbox_inches = 'tight')
    if show is not None: plt.show()
    return

## Monthly Mean Temperature

In [13]:
# Monthly Mean Temperature (Bhutan) w/ statistics
for month in pd.date_range(date_range[0], date_range[1], freq = '1MS'):
    obs = []
    for df in dict_df.values():
        try: obs.append(float(df['Tmean'].loc[month]))
        except: obs.append(np.inf)
    obs = np.concatenate([np.stack(list(station_lonlat.values())), np.array(obs)[:, None]], axis = 1)
    list_da = []
    for da in list_da_mtmean:
        if month in da['time']: list_da.append(da.sel(time = month))
        else: list_da.append(da.isel(time = 0) * np.inf)
    
    plot_models(list_da = list_da, list_name = list_name_temp,
                list_shapes = [shape_feature, shape_feature_mask], bounds = bounds_bhutan,
                norm = prcp_norm_tminmax, cmap = 'nipy_spectral', suptitle = month.strftime('%Y-%m'), clabel = 'Monthly Mean Temperature [degC]',
                stats = True, stats_polygon = polygon_feature, stats_unit = 'degC',
                savefig = os.path.join(path_figs, f'HEROES-TMEAN-{month.strftime("%Y-%m")}-Bhutan.png'), show = None,
                obs = obs)
    plt.close('all')

/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245

## Monthly Minimum Temperature

In [14]:
# Monthly Minimum Temperature (Bhutan) w/ statistics
for month in pd.date_range(date_range[0], date_range[1], freq = '1MS'):
    obs = []
    for df in dict_df.values():
        try: obs.append(float(df['Tmin'].loc[month]))
        except: obs.append(np.inf)
    obs = np.concatenate([np.stack(list(station_lonlat.values())), np.array(obs)[:, None]], axis = 1)
    list_da = []
    for da in list_da_mtmin:
        if month in da['time']: list_da.append(da.sel(time = month))
        else: list_da.append(da.isel(time = 0) * np.inf)
    
    plot_models(list_da = list_da, list_name = list_name_temp,
                list_shapes = [shape_feature, shape_feature_mask], bounds = bounds_bhutan,
                norm = prcp_norm_tminmax, cmap = 'nipy_spectral', suptitle = month.strftime('%Y-%m'), clabel = 'Monthly Minimum Temperature [degC]',
                stats = True, stats_polygon = polygon_feature, stats_unit = 'degC',
                savefig = os.path.join(path_figs, f'HEROES-TMIN-{month.strftime("%Y-%m")}-Bhutan.png'), show = None,
                obs = obs)
    plt.close('all')

/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245

## Monthly Maximum Temperature

In [15]:
# Monthly Maximum Temperature (Bhutan) w/ statistics
for month in pd.date_range(date_range[0], date_range[1], freq = '1MS'):
    obs = []
    for df in dict_df.values():
        try: obs.append(float(df['Tmax'].loc[month]))
        except: obs.append(np.inf)
    obs = np.concatenate([np.stack(list(station_lonlat.values())), np.array(obs)[:, None]], axis = 1)
    list_da = []
    for da in list_da_mtmax:
        if month in da['time']: list_da.append(da.sel(time = month))
        else: list_da.append(da.isel(time = 0) * np.inf)
    
    plot_models(list_da = list_da, list_name = list_name_temp,
                list_shapes = [shape_feature, shape_feature_mask], bounds = bounds_bhutan,
                norm = prcp_norm_tminmax, cmap = 'nipy_spectral', suptitle = month.strftime('%Y-%m'), clabel = 'Monthly Maximum Temperature [degC]',
                stats = True, stats_polygon = polygon_feature, stats_unit = 'degC',
                savefig = os.path.join(path_figs, f'HEROES-TMAX-{month.strftime("%Y-%m")}-Bhutan.png'), show = None,
                obs = obs)
    plt.close('all')

/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245.py:27: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect = [0, 0.05, 1, 1])
/tmp/ipykernel_7881/3200789245